In [125]:

from sklearn.feature_extraction.text import CountVectorizer  # 피처 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

In [127]:
import pandas as pd
review_jp =pd.read_csv('건대_군자_광진_일식_review_list.csv')
review_kr=pd.read_csv('건대_군자_광진_한식_review_list.csv')
review_chn=pd.read_csv('건대_군자_광진_중식_review_list.csv')




In [128]:
review_all=pd.concat([review_jp,review_kr,review_chn])

In [129]:
review_all['맛평가'].unique()

array(['맛있다', '괜찮다', '별로'], dtype=object)

In [130]:
review_all['가게이름'].unique()

array(['초라멘', '소바쿠', '라멘다이야', '포비', '부탄츄', '마리모', '유미카츠', '스시붐', '노다메',
       '겐로쿠우동', '멘쇼', '홍대돈부리', '끼니밥메시', '라멘미세', '소정백', '고씨네', '플레져',
       '오코노미야키식당하나', '이이요', '케루악', '우마이도', '요마시', '초밥짓는원숭이', '건대스시', '호야',
       '쉐프의콜렉션', '오사이초밥', '데판야끼 한', '코코이찌방야', '피읖', '포비의 행방불명', '스시오리',
       '심야식당7925', '백소정', 'D 27', '고베규카츠', '팬더라이스', '일광', '와꼬스시', '달빛한점',
       '오징어상미수산', '원조할아버지손두부', '이화만두', '서북면옥', '궤도에오르다', '콩이네', '은혜떡볶이',
       '또래끼리', '골목에골목', '돕감자탕전문점', '신토불이떡볶이', '어멍네고기국수', '아찌떡볶이', '사대분식',
       '송림식당', '망향비빔국수', '민속손국시', '콩불', '겨울나그네', '스위트앤카츠', '신선설농탕',
       '할머니순대국', '올래상회', '정면', '뉴욕떡볶이', '대원칼국수', '알촌', '찡떡', '낙곱집',
       '포크포크', '남한강민물매운탕', '환이네갈비살 (휴업중)', '태천면옥', '계탄집', '장인석닭한마리',
       '고기굽는놈', '민정식당', '쭈꾸미킹', '뱃놈', '왕십리정곱창', '김돈이', '최신족발', '순곱이네',
       '강릉엄지네꼬막집', '신사소곱창', '바른치킨', '양촌리생삼겹살', '등촌샤브칼국수', '봉추찜닭', '불난닭발',
       '개미집', '강쇠네', '대명소곱창', '정통집', '영미오리탕', '독도수산', '선장훅', '명월관', '시홍쓰',
       '송화산시도삭면', '빠오즈푸', '천향마라탕', '밍지황먼지찜닭', '미식반점', '백미원', '중경소면

In [131]:
len(review_all['가게이름'].unique())

207

In [132]:
review_all['가게이름'].value_counts()[:10]

신토불이떡볶이       127
매운향솥          118
우마이도          112
매화반점          110
이이요           101
서북면옥           95
빠오즈푸           90
명봉샤브샤브양꼬치      86
오코노미야키식당하나     66
해룡마라룽샤         57
Name: 가게이름, dtype: int64

In [133]:
dict(review_all['가게이름'].value_counts()[:10])

{'신토불이떡볶이': 127,
 '매운향솥': 118,
 '우마이도': 112,
 '매화반점': 110,
 '이이요': 101,
 '서북면옥': 95,
 '빠오즈푸': 90,
 '명봉샤브샤브양꼬치': 86,
 '오코노미야키식당하나': 66,
 '해룡마라룽샤': 57}

In [142]:
most=dict(review_all['가게이름'].value_counts()[:20])

In [143]:
keys =[*most]
keys

['신토불이떡볶이',
 '매운향솥',
 '우마이도',
 '매화반점',
 '이이요',
 '서북면옥',
 '빠오즈푸',
 '명봉샤브샤브양꼬치',
 '오코노미야키식당하나',
 '해룡마라룽샤',
 '복만루',
 '골목에골목',
 '돕감자탕전문점',
 '궤도에오르다',
 '송화산시도삭면',
 '노다메',
 '원조할아버지손두부',
 '봉자마라탕',
 '은혜떡볶이',
 '화양식당']

In [145]:
import numpy
import re

def REVIEW_Concat_Str(review, food_type):
    #리뷰를 가게 1개 각각 기준으로 합치기, 영어/숫자/특수문자 제거
    review_all =[]
    for i in range(len(review['가게이름'].unique())):
        li=str(review[review['가게이름']==review['가게이름'].unique()[i]]['리뷰'])
        
        review_all.append(li)

    df ={
        'name' : review['가게이름'].unique(),
        'review' :review_all,
        'cate': food_type
    }

    review_concat =pd.DataFrame(df)
    
    review_concat['review']=[re.sub('[0-9A-Za-z-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》_]', '', str) for str in review_concat['review']]
    # re.sub('[^A-Za-z0-9가-힣]', '', string)
    return review_concat
    

In [146]:
review_concat_kr=REVIEW_Concat_Str(review_kr, '한식')
review_concat_jp=REVIEW_Concat_Str(review_jp, '일식')
review_concat_chn=REVIEW_Concat_Str(review_chn, '중식')


In [147]:
review=pd.concat([review_concat_kr,review_concat_jp,review_concat_chn])

In [148]:
review

,name,review,cate
0,원조할아버지손두부,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집 늘 대기가 조금...,한식
1,이화만두,만두계의 강호동 덩치는 엄청 큰데 속은 왜 이렇게 부드러운거야이미 유명하고도...,한식
2,서북면옥,서울특별시 광진구 구의동에 위치한 서북면옥에서 물냉면천원과 접시만두 조각\...,한식
3,궤도에오르다,기본에 충실해서 더 맛있었던 핫도그였습니다 들어가서 음식이 나오기까지는 꽤 ...,한식
4,콩이네,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다위...,한식
...,...,...,...
104,해란양꼬치,양꼬치 옥수수면 감자 소고기를 시켰다 양고기는 서울 다른 곳에서 먹는 것보다...,중식
105,송쉐프,전가복도 맛있네욤이번에도 연태구냥과 볶음...,중식
106,원보양꼬치,건대입구에는 유명한 양꼬치골목이 있다 양꼬치집 가게들도 많은데 자주가는 몇 ...,중식
107,금룡,...,중식


In [149]:
review[review['name'].isin(keys)]

,name,review,cate
0,원조할아버지손두부,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집 늘 대기가 조금...,한식
2,서북면옥,서울특별시 광진구 구의동에 위치한 서북면옥에서 물냉면천원과 접시만두 조각\...,한식
3,궤도에오르다,기본에 충실해서 더 맛있었던 핫도그였습니다 들어가서 음식이 나오기까지는 꽤 ...,한식
5,은혜떡볶이,와 여기 진짜 대학생 때부터 너무 가보고 싶었던 곳인데 이제서야 와본거 실화...,한식
7,골목에골목,깔끔하고 맛있고 양도 푸짐하다사장님 고생하시는거 같아 마...,한식
8,돕감자탕전문점,맛집인스타 건대에서 감자탕 맛집으로 손꼽히는 곳 지하까\n ...,한식
9,신토불이떡볶이,떡볶이를 너무 좋아하시는 분들이라면 모를 수 없는 아차산 신토불이 떡볶이에요...,한식
8,노다메,어린이대공원역 정갈한 카레 한상새우카레가 제일 맛있다새우튀김은 추가하여 \...,일식
17,오코노미야키식당하나,정성이 가득 담긴 오코노미야키광진구에 있는 오코노미야키 가게 오코노미야키식당...,일식
18,이이요,군자에 위치한 덮밥 성지인스타는 물론 다양한 매체에서 눈을 자극하며가보고 싶...,일식


In [84]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(review['cate']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [150]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(review['review']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [151]:
place_simi_co = (
                 place_simi_cate * 0.1 # 공식 1. 카테고리 유사도
                + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                 #+ np.repeat([df['kakao_blog_review_qty'].values], len(df['kakao_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                 #+ np.repeat([df['kakao_star_point'].values], len(df['kakao_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                 #+ np.repeat([df['kakao_star_point_qty'].values], len(df['kakao_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )



place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]




In [152]:
# place_simi_co = (
#                   place_simi_cate * 0.0 # 카테고리 유사도
#                 + place_simi_review * 1 # 리뷰 텍스트 유사도
#                  #+ np.repeat([df['kakao_blog_review_qty'].values], len(df['kakao_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
#                  #+ np.repeat([df['kakao_star_point'].values], len(df['kakao_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
#                  #+ np.repeat([df['kakao_star_point_qty'].values], len(df['kakao_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
#                  )

In [153]:
place_simi_co = (
                  place_simi_cate * 0.3 # 카테고리 유사도
                + place_simi_review * 1 # 리뷰 텍스트 유사도
                + np.repeat([review['point']], len(review['point']) , axis=0) * 0.005
                #평가가 얼마나 좋은지
                + np.repeat([review['qty'].values], len(review['qty']) , axis=0) * 0.001
                 #+리뷰가 얼마나 많은지
                 )

In [155]:
find_simi_place(review, place_simi_co_sorted_ind, '신토불이떡볶이', 5)

,name,review,cate
9,신토불이떡볶이,떡볶이를 너무 좋아하시는 분들이라면 모를 수 없는 아차산 신토불이 떡볶이에요...,한식
7,골목에골목,깔끔하고 맛있고 양도 푸짐하다사장님 고생하시는거 같아 마...,한식
25,쉐프의콜렉션,돌아가는 길에 다음 예약 잡고 싶구요너무 맛있습니다 숙성회 처음에 한입거리\...,일식
25,춘선만두,엉엉엉엉엉엉엉엉이 집 묘한 매력이 있다 향신료가 강해 망설여지나어느새인가 흡...,중식
0,원조할아버지손두부,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집 늘 대기가 조금...,한식


In [156]:
find_simi_place(review, place_simi_co_sorted_ind, '이이요', 5)

,name,review,cate
18,스위트앤카츠,저만 몰랐나요 어린이대공원에 돈까스 무한리필이 있다는거 심지어 함께 이용할...,한식
35,민정식당,꽤 오래전부터 가고싶었는데 이제야 방문넷이서 수육큰것 돈까스 주문수육 양도 ...,한식
21,요마시,지나가다 그냥 들어간 거 였는데 주차도 쉽고 음식도 맛있었어요 돈까스 전문점...,일식
82,구구향훠궈,오늘의업보 협찬무한리필 꿔바로우로 반응이 좋았던 영상 구구향훠궈건대점 하지만...,중식
6,유미카츠,키오스크로 주문해서 셀프로 찾아먹는 돈까스 전문점이에요 셀프 시스템이지만 내...,일식


In [157]:
find_simi_place(review, place_simi_co_sorted_ind, '매화반점', 5)

,name,review,cate
7,스시붐,맛이 좋았지만 기대에 비해서는 조금 아쉬운 곳이었습니다 이 곳의 대표메뉴인...,일식
0,원조할아버지손두부,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집 늘 대기가 조금...,한식
40,오징어상미수산,연어회가 꽃처럼 피었습니다신선하고 윤기좌르르 흐르는 도톰한 연어를가성비 좋은...,일식
12,끼니밥메시,요즘 날이 더워지다 보니 뭔가 시원한 게 당기는데 내가 좋아하는 여기 끼니밥...,일식
13,동네수타짜장,해물가득해요 짜장짱맛 수타중에 최고에용\n 리뷰,중식


In [158]:
find_simi_place(review, place_simi_co_sorted_ind, '우마이도', 5)

,name,review,cate
20,할머니순대국,대박 태어나서 이렇게 건더기 많은 순대국집 처음가봤다나는 서울사람이 아니고 ...,한식
60,화양식당,도삭면 짬뽕을 못잊어 재재재방문한 화양식당 이번엔 제일 유명하다는 유린기중도...,중식
17,하오츠,음 군자에서 제일 괜찮은 중국집이라해서 갔는데 간이 전반적으로 넘 심심해서 ...,중식
7,골목에골목,깔끔하고 맛있고 양도 푸짐하다사장님 고생하시는거 같아 마...,한식
34,D 27,얼마전에 다녀온 군자술집 이라는 곳은 군자역에서도 세종대와 근접하게 위차한...,일식


In [170]:
find_simi_place(review, place_simi_co_sorted_ind, '서북면옥', 5)

,name,review,cate
2,서북면옥,서울특별시 광진구 구의동에 위치한 서북면옥에서 물냉면천원과 접시만두 조각\...,한식
31,태천면옥,지금 능라도강남점삼성동이 역삼동에 있을때 그곳에서 주방 면장을 하셨던 분이 ...,한식
40,오징어상미수산,연어회가 꽃처럼 피었습니다신선하고 윤기좌르르 흐르는 도톰한 연어를가성비 좋은...,일식
85,중경삼림,무나리뷰 둘 다 쥐긴 쉽지않을껄건대 커먼그라운드 층에 위치한 중경삼림은 오픈...,중식
66,청두인상,요약 쏸차이위를 찾아 떠났던 곳이지만 알고보니 사천음식 전부 잘하는 식당 고...,중식


In [171]:
find_simi_place(review, place_simi_co_sorted_ind, '아찌떡볶이', 5)

,name,review,cate
11,아찌떡볶이,딱 아는 떡볶이 맛이지만 계속해서 손이가는 맛떡이 길고 얇아 오물오물 씹어먹...,한식
7,골목에골목,깔끔하고 맛있고 양도 푸짐하다사장님 고생하시는거 같아 마...,한식
23,뉴욕떡볶이,년째 자주가는 동네떡볶이집가격도 착하고 맛있고 최고에요단골집 잘되길 바라는마...,한식
4,콩이네,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다위...,한식
20,할머니순대국,대박 태어나서 이렇게 건더기 많은 순대국집 처음가봤다나는 서울사람이 아니고 ...,한식
